In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import argparse
import secrets
import json
import sys
import math 

In [3]:
import sys
sys.path.append('/usr0/home/naveenr/projects/patient_provider')

In [4]:
from patient.simulator import run_multi_seed
from patient.baseline_policies import *
from patient.lp_policies import *
from patient.utils import get_save_path, delete_duplicate_results, restrict_resources, one_shot_policy, MyEncoder

In [5]:
is_jupyter = 'ipykernel' in sys.modules

In [224]:
if is_jupyter: 
    seed        = 43
    num_patients = 20
    num_providers = 20
    provider_capacity = 1
    noise = 0.25
    fairness_constraint = -1
    num_trials = 20
    utility_function = "normal"
    order="uniform"
    online_arrival = False  
    new_provider = False 
    out_folder = "dynamic"
    average_distance = 20.2
    max_shown = 5
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', help='Random Seed', type=int, default=42)
    parser.add_argument('--n_patients',         '-N', help='Number of patients', type=int, default=100)
    parser.add_argument('--n_providers',        help='Number of providers', type=int, default=100)
    parser.add_argument('--max_shown',        help='How many providers to show at most', type=int, default=25)
    parser.add_argument('--provider_capacity', help='Provider Capacity', type=int, default=1)
    parser.add_argument('--noise', help='Noise in theta', type=float, default=0.1)
    parser.add_argument('--average_distance', help='Maximum distance patients are willing to go', type=float, default=20.2)
    parser.add_argument('--fairness_constraint', help='Maximum difference in average utility between groups', type=float, default=-1)
    parser.add_argument('--num_trials', help='Number of trials', type=int, default=100)
    parser.add_argument('--utility_function', help='Which folder to write results to', type=str, default='uniform')
    parser.add_argument('--order', help='Which folder to write results to', type=str, default='uniform')
    parser.add_argument("--online_arrival",action="store_true",help="Patients arrive one-by-one")
    parser.add_argument("--new_provider",action="store_true",help="Are we simulating a new provider matching")
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='policy_comparison')

    args = parser.parse_args()

    seed = args.seed
    num_patients = args.n_patients
    num_providers = args.n_providers 
    num_trials = args.num_trials
    noise = args.noise
    average_distance = args.average_distance
    fairness_constraint = args.fairness_constraint
    provider_capacity = args.provider_capacity
    max_shown = args.max_shown
    utility_function = args.utility_function
    order = args.order
    online_arrival = args.online_arrival
    new_provider = args.new_provider 
    out_folder = args.out_folder
    
assert not(online_arrival and new_provider)
save_name = secrets.token_hex(4)  

In [226]:
results = {}
results['parameters'] = {'seed'      : seed,
        'num_patients'    : num_patients,
        'num_providers': num_providers, 
        'provider_capacity'    : provider_capacity,
        'max_shown': max_shown,
        'utility_function': utility_function, 
        'order': order, 
        'num_trials': num_trials, 
        'noise': noise, 
        'average_distance': average_distance,
        'online_arrival': online_arrival,
        'new_provider': new_provider,
        'fairness_constraint': fairness_constraint} 

## Baselines

In [227]:
seed_list = [seed]
restrict_resources()

In [237]:
policy = one_shot_policy
per_epoch_function = offer_everything
name = "offer_everything"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['offer_everything_num_matches'])/num_patients,np.mean(results['offer_everything_patient_utilities'])))

offer_everything policy
Matches 0.26749999999999996, Utilities 0.8786481895447724


In [235]:
policy = one_shot_policy
per_epoch_function = greedy_policy
name = "greedy"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

greedy policy
Matches 0.34500000000000003, Utilities 0.8954628759602733


In [234]:
policy = one_shot_policy
if fairness_constraint != -1:
    per_epoch_function = get_fair_optimal_policy(fairness_constraint,seed)
else:
    per_epoch_function = optimal_policy
name = "omniscient_optimal"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function,use_real=True)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

omniscient_optimal policy
Matches 0.365, Utilities 0.930388057039007


## Optimization-Based

In [233]:
policy = one_shot_policy
per_epoch_function = lp_policy
name = "lp"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

lp policy
Matches 0.21000000000000002, Utilities 0.8824845759590648


In [238]:
policy = one_shot_policy
if fairness_constraint != -1:
    per_epoch_function = get_gradient_policy(num_patients,num_providers,fairness_constraint,seed)
else:
    per_epoch_function = gradient_policy
name = "gradient_descent"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

gradient_descent policy
Set parameter LogToConsole to value 0
0.9199007870548277
0.9186216871844876 0.07191148815912807 [18.61946181 17.9816358  18.31190248 18.65010894 18.21601964 18.65678651
 18.58926476 18.94100263 18.82793056 18.35948302 17.80941534 18.79446618
 17.93400765 18.48058939 17.89222509 18.48603618 18.43599682 18.13213975
 18.14991755 18.18028478]
Set parameter LogToConsole to value 0
{'feasible': False}
0.8923574917170531
Matches 0.35750000000000004, Utilities 0.8982319799764131


In [92]:
np.sum(results['greedy_assortments'][0],axis=1)

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [95]:
X_greedy = np.array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1]])

In [154]:
def simulate_from_X(X, weights_list, capacities_list, permutations):
    """
    Produces y[i,j,k], z[i,k], c[t,j,k] that match the MILP shapes.
    
    X: (N, M)
    weights_list: list of (N, M+1) arrays
    capacities_list: list of length M (real options only)
    permutations: list of permutations (one per scenario)

    Returns:
        y_sim: (N, M+1, K)
        z_sim: (N, K)
        c_sim: (N, M, K)
    """

    K = len(weights_list)
    N, M_plus1 = weights_list[0].shape
    M = M_plus1 - 1  # exit option index is M

    y_sim = np.zeros((N, M_plus1, K), dtype=int)
    z_sim = np.zeros((N, K))
    c_sim = np.zeros((N, M, K))

    for k in range(K):
        weights = weights_list[k]
        perm = permutations[k]
        cap = capacities_list.copy()

        for t, i in enumerate(perm):
            
            # c[t,j,k] values BEFORE this patient chooses
            for j in range(M):
                if t == 0:
                    c_sim[t, j, k] = cap[j]
                else:
                    # c at time t is after previous patient's update
                    c_sim[t, j, k] = c_sim[t-1, j, k] - y_sim[perm[t-1], j, k]

            # compute availability for this patient
            available = np.zeros(M_plus1)
            for j in range(M):
                if X[i, j] == 1 and c_sim[t, j, k] > 0:
                    available[j] = 1
            available[M] = 1  # exit always allowed


            # choose argmax over available weights
            choices = np.where(available == 1)[0]
            j_sel = choices[np.argmax(weights[i, choices])]
            if i == 3:
                print("Choices {}, j {}".format(choices,j_sel))
                print("Row {}, capacity {}".format(X[i],c_sim[t,:,0]))

            # assign y
            y_sim[i, j_sel, k] = 1
            z_sim[i, k] = weights[i, j_sel]

            # update capacity AFTER choosing
            if j_sel < M:
                cap[j_sel] -= 1

    return y_sim, z_sim, c_sim


In [96]:
permutations = [np.arange(20)]
capacities = [1 for i in range(20)]
weights_list = [weights]

In [184]:

def evaluate_fixed_X(X_sol, weights_list, capacities, permutations):
    N, M = X_sol.shape
    M_plus1 = M + 1
    K = len(weights_list)

    model = gp.Model("evaluate_fixed_X")
    model.Params.LogToConsole = 0

    # Variables (same as before)
    y = model.addVars(N, M_plus1, K, vtype=GRB.BINARY, name="y")
    z = model.addVars(N, K, vtype=GRB.CONTINUOUS, name="z")
    c = model.addVars(N, M, K, vtype=GRB.CONTINUOUS, name="c")

    for k in range(K):
        weights_k = weights_list[k]
        ordering = permutations[k]

        # One selection per patient
        for i in range(N):
            model.addConstr(gp.quicksum(y[i,j,k] for j in range(M_plus1)) == 1)

        # Sequence + capacities
        for t in range(N):
            patient = ordering[t]

            for j in range(M):
                if t == 0:
                    # initial capacity
                    model.addConstr(c[t,j,k] == capacities[j])
                else:
                    prev_patient = ordering[t-1]
                    model.addConstr(c[t,j,k] == c[t-1,j,k] - y[prev_patient,j,k])

                # availability constraints
                if X_sol[patient, j] == 0:
                    model.addConstr(y[patient,j,k] == 0)
                else:
                    model.addConstr(y[patient,j,k] <= c[t,j,k])  # capacity >0 if chosen

            # exit option always available
            pass

        # link utility
        for i in range(N):
            model.addConstr(z[i,k] == gp.quicksum(weights_k[i,j] * y[i,j,k]
                                                  for j in range(M_plus1)))

        # rationality (Big-M)
        bigM = 1
        for t in range(N):
            patient = ordering[t]
            for j in range(M_plus1):
                for l in range(M_plus1):
                    if j == l:
                        continue
                    if l < M:
                        if X_sol[patient, l] == 1:
                            model.addConstr(
                                z[patient,k] >= weights_k[patient,l]
                                            - bigM*(1 - y[patient,j,k])
                                            - bigM*(1 - X_sol[patient,l])
                                            - bigM*(1 - c[t,l,k])   # effectively disables if capacity = 0
                            )
                    else:
                        # exit always available
                        model.addConstr(z[patient,k] >=
                            weights_k[patient,l] -
                            bigM * (1 - y[patient,j,k]))

    # objective value for fixed X
    model.setObjective(gp.quicksum(z[i,k]
                        for i in range(N) for k in range(K)), GRB.MAXIMIZE)

    model.optimize()

    # Extract y, z, c, objective
    y_sol = np.zeros((N, M_plus1, K))
    z_sol = np.zeros((N, K))
    c_sol = np.zeros((N, M, K))

    feasible = model.status == GRB.OPTIMAL

    if feasible:
        for i in range(N):
            for k in range(K):
                z_sol[i,k] = z[i,k].X
                for j in range(M_plus1):
                    y_sol[i,j,k] = y[i,j,k].X
            for j in range(M):
                for k in range(K):
                    c_sol[i,j,k] = c[i,j,k].X

        return {
            "feasible": True,
            "objective": model.ObjVal,
            "y": y_sol,
            "z": z_sol,
            "c": c_sol,
        }
    else:
        return {"feasible": False}

In [187]:
def check_MILP_constraints(X, y, z, c, weights_list, capacities, permutations, max_shown=None):
    K = len(weights_list)
    N, M_plus1, _ = y.shape
    M = M_plus1 - 1

    for k in range(K):
        weights_k = weights_list[k]
        ordering = permutations[k]

        # One selection per patient
        for i in range(N):
            assert sum([y[i,j,k] for j in range(M_plus1)]) == 1
            # model.addConstr(gp.quicksum(y[i,j,k] for j in range(M_plus1)) == 1)

        # # Sequence + capacities
        for t in range(N):
            patient = ordering[t]

            for j in range(M):
                if t == 0:
                    # initial capacity
                    assert c[t,j,k] == capacities[j]
                else:
                    prev_patient = ordering[t-1]
                    assert (c[t,j,k] == c[t-1,j,k] - y[prev_patient,j,k])

        #         # availability constraints
                if X[patient, j] == 0:
                    assert y[patient,j,k] == 0
                else:
                    assert y[patient,j,k] <= c[t,j,k]  # capacity >0 if chosen

        #     # exit option always available
            pass

        # # link utility
        for i in range(N):
            assert (z[i,k] == sum(weights_k[i,j] * y[i,j,k]
                                                  for j in range(M_plus1)))

        # # rationality (Big-M)
        bigM = 1
        for t in range(N):
            patient = ordering[t]
            for j in range(M_plus1):
                for l in range(M_plus1):
                    if j == l:
                        continue
                    if l < M:
                        if X[patient, l] == 1 and c[patient,l,0] == 1:
                            assert(
                                z[patient,k] >= weights_k[patient,l]
                                            - bigM*(1 - y[patient,j,k])
                                            - bigM*(1 - X[patient,l])
                                            - bigM*(1 - c[t,l,k])   # effectively disables if capacity = 0
                            )
        #                 # exit always available
        #                 model.addConstr(z[patient,k] >=
        #                     weights_k[patient,l] -
        #                     bigM * (1 - y[patient,j,k]))



In [177]:
y,z,c = simulate_from_X(X_greedy, [weights],capacities,permutations)

Choices [ 9 11 20], j 20
Row [0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1], capacity [1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0.]


In [188]:
check_MILP_constraints(X_greedy, y, z, c, [weights], capacities, permutations)

In [191]:
evaluate_fixed_X(X_greedy,[weights],capacities,permutations)

Set parameter LogToConsole to value 0


{'feasible': True,
 'objective': 18.14174163716393,
 'y': array([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
       

In [121]:
z.shape

(21,)

In [120]:
check_constraints(X_greedy, y, z, capacities, weights_list[0], permutations)

(False,
 'z mismatch: z=[0 1 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 1 0], sum(y real providers)=[0 1 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 1]')

## Save Data

In [46]:
save_path = get_save_path(out_folder,save_name)

In [47]:
delete_duplicate_results(out_folder,"",results)

In [48]:
json.dump(results,open('../../results/'+save_path,'w'),cls=MyEncoder)